# 0.1 Extract data

This notebook extracts the data availability statements from the Wiley data warehouse.

In [1]:
import os

import pandas as pd
from sqlalchemy import create_engine, engine, MetaData, Table, select, or_
from snowflake.sqlalchemy import URL
from dotenv import load_dotenv, find_dotenv

In [3]:
# Get database credentials
load_dotenv(find_dotenv())

SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_ROLE = os.getenv("SNOWFLAKE_ROLE")
SNOWFLAKE_DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA = os.getenv("SNOWFLAKE_SCHEMA")
SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")

In [12]:
# Create database engine

engine = create_engine(
    URL(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        role=SNOWFLAKE_ROLE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        warehouse=SNOWFLAKE_WAREHOUSE,
    )
)

In [ ]:
# Connect and reflect tables

conn = engine.connect()
metadata = MetaData()

questions = Table(
    "S1_ARTICLES_SUBMISSION_CUSTOM_QUESTIONS",
    metadata,
    autoload=True,
    autoload_with=engine,
)
articles = Table("S1_ARTICLES", metadata, autoload=True, autoload_with=engine)

In [ ]:
%%time

# Get data

stmt = (
    select(
        [
            questions.c.site_name,
            questions.c.documentid,
            questions.c.customquestionid,
            questions.c.questiontext,
            questions.c.submissioncustomanswer_answertext.label("answertext"),
            articles.c.submissiondate,
        ]
    )
    .select_from(
        questions.join(articles, questions.c.documentid == articles.c.documentid)
    )
    .where(
        or_(
            questions.c.questiontext.ilike("%data%"),
            questions.c.submissioncustomanswer_answertext.ilike("%data%"),
        )
    )
)

df = pd.read_sql(stmt, conn)

In [ ]:
df.info()

In [ ]:
PROJ_ROOT = os.path.join(os.pardir)
save_path = os.path.join(PROJ_ROOT + '/data/raw/' + 'das.feather')

df.to_feather(save_path)